# Kaggle
## Competição DSA de Machine Learning - Dezembro 2019

Versão 1.0.9: LB = ??? CV = 0.464466 / CV com calibration = 
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado pelo Sandro
- usando calibrateCV

In [1]:
# Import libraries to store data
import pandas as pd
import numpy as np
# Import libraries to visualize data
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from ipywidgets import interact, interact_manual
# Import libraries to process data
import tsfresh
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Import libraries to classify data and score results
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score
import xgboost as xgb
# Import libraries used in functions and for feedback
import time
import random
import warnings
warnings.filterwarnings("ignore") # Setting values on a Pandas DataFrame 
# sometimes throws errors, so I'll silence the warnings.

# The below has to be set because matplotlib and XGB crash 
# by causing a duplicate copy of a library to be run. It's a 
# known issue, unfortunately.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

import time
import datetime
import gc

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

In [3]:
# Importa os pacotes de algoritmos
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier

# Importa pacotes do sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler
from sklearn.calibration import CalibratedClassifierCV

# Importa pacotes do sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder


from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

In [4]:
def read_data():
    
    print('Carregando arquivo dataset_treino.csv....')
    train = pd.read_csv('../dataset/dataset_treino.csv')
    print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))

    print('Carregando arquivo dataset_teste.csv....')
    test = pd.read_csv('../dataset/dataset_teste.csv')
    print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))

    return train, test

# Leitura dos dados
train, test = read_data()

df = train.append(test)


Carregando arquivo dataset_treino.csv....
dataset_treino.csv tem 114321 linhas and 133 colunas
Carregando arquivo dataset_teste.csv....
dataset_teste.csv tem 114393 linhas and 132 colunas


In [5]:
df.head()

,ID,target,v1,v10,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v11,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v12,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v13,v130,v131,v14,v15,v16,v17,v18,v19,v2,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v3,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v4,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v5,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v6,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v7,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v8,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v9,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,3,1.0,1.335739,0.503281,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,16.434108,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,6.085711,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,2.866830,0.636365,2.857144,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,8.727474,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,3.921026,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,7.915266,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,2.599278,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,3.176895,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,0.012941,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,9.999999,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337
1,4,1.0,NaN,1.312910,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,NaN,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,6.507647,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,9.191265,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,2.301630,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN
2,5,1.0,0.943877,0.765864,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,14.756098,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,6.384670,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,2.505589,0.883118,1.176472,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,5.310079,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,C,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,4.410969,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,5.326159,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,3.979592,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,3.928571,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,0.019645,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,12.666667,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837
3,6,1.0,0.797415,6.542669,18.256352,8.507281,2.503055,4.872157,2.573664,0.113967,12.554274,B,2.230754,1.990131,16.347483,B,2.643678,J,NaN,13.777666,10.574713,1.511063,4.949609,7.180722,5.655086e-01,9.646653,1.166281,1.956521,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,3.903302,1.677108,1.034483,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,8.304757,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,4.225930,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,C,12.194855,8.591954,11.627438,3.329176,4.780357,H,16.621695,0.139721,1.178161,BW,3.

In [6]:
# Verificar a quantidade de features numericas e categoricas

numerical_feats = df.dtypes[df.dtypes != "object"]
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = df.dtypes[df.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  114
Number of Categorical features:  19


In [7]:
# Transformando as features categorias com LabelEncoder
le = LabelEncoder()
for i, col in enumerate(df):
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(np.array(df[col].astype(str)).reshape((-1,)))
        
        
# Realizando tratamento de missing value
for c in df.columns:
    if c != 'ID' and c != 'target':
        df[c].fillna(df[c].mean(),inplace=True)
        
scaler = StandardScaler()
for c in df.columns:
    col_type = df[c].dtype
    #if col_type == 'float64' and c != 'target':
    if c != 'ID' and c != 'target':
        df[c] = scaler.fit_transform(df[c].values.reshape(-1, 1))

<a id='section5'></a>
# Feature Extraction
The next step is to extract features from the signals in order to develop a series of classifiers to classify the remaining data. We've looked at some features above (RMS, PSD), but there are a wealth of other features that I can pull out that are likely meaningful. The package TSFRESH was developed to automate feature extraction and selection from time series data. Their work is published in [this paper linked here](https://www.sciencedirect.com/science/article/pii/S0925231218304843?via%3Dihub), and the packgage documentation is publicly available [here](https://tsfresh.readthedocs.io/en/latest/).

I will only be extracting features from the motion data, since its the only data that should have any predictive power. After all, the position of the robot shouldn't predict what surface it's on (it's not the case that robots only walk east on concrete, though that would make for an interesting story). The position data was exploited above since I knew that the training data was split from a series of longer runs (original groups) and suspected that the same was true for the test data. If I have time, I will analyze how well the model performs in the absence of knowledge from stitching the data. It makes sense to keep the data stitched, however, since it is reasonable to assume that the robot walks on the same surface for at least an extended period of time.

In any case, I'll extract the relevant features below. Open up the collapsed code to see which features I chose to extract.

In [ ]:
extract_from = df[df.columns[2:10]]
extract_from.head()

In [ ]:
extracted_features = tsfresh.extract_features(extract_from,
                                              column_id='v10',
                                              column_sort='v1',
                                              n_jobs = 2)

In [ ]:
efc = extracted_features.copy()
tsfresh.utilities.dataframe_functions.impute(efc)
y = all_stitched.groupby('series_id').max()['surface']
matched_y = y[y!='no_surface']
matched_ef = efc.iloc[matched_y.index]

In [ ]:
features_filtered = tsfresh.select_features(matched_ef, matched_y)

In [ ]:
feature_columns = features_filtered.columns

<a id='section6'></a>
# Classifier Exploration
The purpose of this section is to develop a set of classifiers that I can leverage for a hybrid classifier. It's only worthwhile to keep classifiers that are relatively accurate, but I'll have to explore a series of them in order to choose the best ones. I'll make a train/test (validate) set first in order to assess the classifiers. Then I'll dive headlong into the different kinds of classifiers that are available.

I will intentionally use the unclassified stitched data below to not introduce any extraneous noise into the classifiers. Once the classifiers are trained I can toss them the data that I have yet to classify.

In [ ]:
metadata = all_stitched.groupby('series_id').max()[['surface','test','new_group_id', 'orig_id']]

In [ ]:
filtered_extracted = metadata.join(extracted_features[feature_columns])

In [ ]:
train_filtered = filtered_extracted[filtered_extracted.test == 0]
test_filtered = filtered_extracted[filtered_extracted.test == 1]

In [ ]:
train_filtered.to_csv('data/train_filtered_v2_v7.csv',index=False)
test_filtered.to_csv('data/test_filtered_v2_v7.csv',index=False)

Filling missing NAs and infinite data ∞ by zeroes 0

In [ ]:
train_filtered.fillna(0,inplace=True)
test_filtered.fillna(0,inplace=True)
train_filtered.replace(-np.inf,0,inplace=True)
train_filtered.replace(np.inf,0,inplace=True)
test_filtered.replace(-np.inf,0,inplace=True)
test_filtered.replace(np.inf,0,inplace=True)

In [8]:
treino = df[df['target'].notnull()]
teste = df[df['target'].isnull()]

X_filtered = treino.drop(labels = ['ID','target'], axis = 1)
y_filtered = treino.target

X_f_train, X_f_test, y_f_train, y_f_test = train_test_split(X_filtered, y_filtered, test_size=0.1, random_state=42, stratify = y_filtered)

In [9]:
X_f_train.shape

(102888, 131)

I'll also scale the data since this is helpful for some classifiers. I will have to keep track of whether or not the data that I'm using is scaled.

The first classifier I'll test is the Random Forest Classifier. **I'll use the filtered extracted dataset without scaling.**

In [10]:
RFC = RandomForestClassifier(n_estimators=1000, random_state = 42, n_jobs=-1)
RFC.fit(X_f_train, y_f_train)
y_f_test_RFC = RFC.predict(X_f_test)
print(classification_report(y_f_test, y_f_test_RFC))

              precision    recall  f1-score   support

         0.0       0.65      0.21      0.31      2730
         1.0       0.80      0.96      0.87      8703

    accuracy                           0.78     11433
   macro avg       0.72      0.59      0.59     11433
weighted avg       0.76      0.78      0.74     11433



Now I'll check Naive Bayes. **I'll use the filtered and scaled data.**

In [11]:
from sklearn.naive_bayes import GaussianNB

NB = GaussianNB()
NB.fit(X_f_train, y_f_train)
y_test_NB = NB.predict(X_f_test)
print(classification_report(y_f_test, y_test_NB))

              precision    recall  f1-score   support

         0.0       0.31      0.75      0.43      2730
         1.0       0.86      0.47      0.61      8703

    accuracy                           0.54     11433
   macro avg       0.58      0.61      0.52     11433
weighted avg       0.72      0.54      0.56     11433



How interesting. This one does better at predicting hard_tiles at the expense of predicting carpet. The scores aren't awful, but they're certainly not great.

Now perform linear SVC using a grid search. This grid search is small, so it runs fairly quickly. **I will use the filtered and scaled dataset for this.**

In [ ]:
C_range = np.logspace(-4, 0, 4)
param_grid = dict(C=C_range)
SVM = GridSearchCV(SVC(kernel = 'linear'), 
                    param_grid=param_grid, 
                    n_jobs=-1, 
                    pre_dispatch=2, 
                    cv = 5)
SVM.fit(X_f_train, y_f_train)
y_test_svc = SVM.predict(X_f_test)
print(classification_report(y_f_test, y_test_svc))

Now perform KNN **using the filtered and scaled data**.

In [12]:
KNN = KNeighborsClassifier(10)
KNN.fit(X_f_train,y_f_train)
y_test_knn = KNN.predict(X_f_test)
print(classification_report(y_f_test, y_test_knn))

              precision    recall  f1-score   support

         0.0       0.38      0.19      0.26      2730
         1.0       0.78      0.90      0.84      8703

    accuracy                           0.73     11433
   macro avg       0.58      0.55      0.55     11433
weighted avg       0.69      0.73      0.70     11433



Check a regular decision tree **using the filtered, scaled data**

In [13]:
DT = DecisionTreeClassifier()
DT.fit(X_f_train,y_f_train)
y_test_dt = DT.predict(X_f_test)
print(classification_report(y_f_test, y_test_dt))

              precision    recall  f1-score   support

         0.0       0.35      0.38      0.37      2730
         1.0       0.80      0.78      0.79      8703

    accuracy                           0.69     11433
   macro avg       0.58      0.58      0.58     11433
weighted avg       0.69      0.69      0.69     11433



And finally, perform XGBoost **on the filtered and scaled data.**

In [14]:
param_grid = {
    'max_depth': [3,4,5,6],  # the maximum depth of each tree
    'min_child_weight':np.linspace(0.8,1.2,4),
    'gamma': np.linspace(0,0.2,4),
}

In [15]:
# The below parameters were found using GridSearchCV, using the parameter search basis above.
XGB = xgb.sklearn.XGBClassifier(learning_rate = 0.02,
                                objective = 'binary:logistic',
                                n_estimators = 200,
                                max_depth = 5,
                                min_child_weight = 1.2,
                                subsample=0.8,
                                colsample_bytree = 0.8,
                                gamma = 0.066,
                                n_jobs = 4,
                                nthreads = 1,
                                silent = True,
                                seed = 42)
XGB.fit(X_f_train,y_f_train)
y_test_XGB = XGB.predict(X_f_test)
print(classification_report(y_f_test, y_test_svc))

NameError: name 'y_test_svc' is not defined

<a id='section7'></a>
# Meta Ensemble Classifier
The real utility in stitching groups together is the ability to classify the entire group as one surface based on the results of classifying its constituent series. The easiest way to think of this is that the series vote on the classification of the group. The surface with the most "votes" is used to classify the entire group, similar to how RFC methods work. Furthermore, the results of each classifier above can be used in order to determine the votes for a group! Since the voting occurs as a sum of the number of instances that each surface appears in a group, each classifier can be assigned a relative weight in the voting schema. I'll employ this method to classify the remaining data.

First I'll make a function that generates and joins predictions from the classifier instances.

In [ ]:
def get_predictions(filtered_data, classifiers, scaler):
    warnings.filterwarnings("ignore")
    
    df = filtered_data[['surface','test','new_group_id', 'orig_id']]
    X = filtered_data.drop(labels = ['surface','test','new_group_id','orig_id'], 
                                 axis = 1).values
    X_scaled = scaler.transform(X)
    for name, classifier_dict in classifiers.items():
        clf = classifier_dict['classifier']
        if classifier_dict['scale']:
            prediction = clf.predict(X_scaled)
        else:
            prediction = clf.predict(X)
        df[name] = prediction
    warnings.resetwarnings()
    return df

In [ ]:
classifiers = {'RFC':{'classifier':RFC,'scale': False},
               'NB':{'classifier':NB,'scale':True},
               'SVM':{'classifier':SVM,'scale':True},
               'KNN':{'classifier':KNN,'scale':True},
               'DT':{'classifier':DT,'scale':True},
               'XGB':{'classifier':XGB,'scale':True}}

In [ ]:
multi_predictions = get_predictions(filtered_data=train_filtered,
                              classifiers=classifiers, 
                              scaler=scaler_f)

The classifier-specific predictions can be used in an ensemble classifier function that is written below. The function accepts the predictions and a dictionary that includes the classifiers to use and their weights.

In [ ]:
def classifier_ensemble(predictions, classifier_weights, groupby = 'new_group_id'):
    warnings.filterwarnings("ignore")
    df = predictions[['test','new_group_id','orig_id']]
    df['surface'] = 'None'
    g = predictions.groupby(groupby)
    for name, group in g:
        index = group.index
        weighted_value_counts = []
        for classifier,weight in classifier_weights.items():
            value_counts = group[classifier].value_counts()
            weighted_value_counts.append(value_counts*weight)
        all_counts = pd.DataFrame(weighted_value_counts).fillna(value=0)
        surface = all_counts.sum().sort_values(ascending=False).index[0]
        df.loc[index,'surface'] = surface
    warnings.resetwarnings()
    return df

In [ ]:
classifier_weights = {classifier:1 for classifier in classifiers.keys()}
del classifier_weights['NB'] # Naive Bayes classification wasn't great, so I won't include it
ensemble_prediction = classifier_ensemble(predictions=multi_predictions,
                                 classifier_weights=classifier_weights)

In [ ]:
print(classification_report(train_filtered.surface, ensemble_prediction.surface))

I'll have to perform my own parameter search on the classifier ensemble in order to find the weights that produce the best results. Using a grid approach might not be the best course of action since there are 5 parameters to set. Instead, I'll use an evolutionary algorithm with a maximum number of generations.

In [ ]:
def CEParamSearch(y_true, multi_predictions, classifier_names, 
                  groupby = 'new_group_id', generations = 20, 
                  seed = 1, mutation_rate = 0.35):  
    # Generate random weights first
    np.random.seed(seed)
    weights = np.random.rand(len(classifiers))
    classifier_weights = {classifier:weight for classifier,weight in zip(classifier_names,weights)}
    
    # Classify the data and calculate a score with the weights
    ensemble_prediction = classifier_ensemble(multi_predictions,classifier_weights,groupby=groupby)
    y_pred = ensemble_prediction.surface
    score = sum(y_pred == y_true)/len(y_pred)
    
    # Keep track of score and time
    scores = [score]
    start = time.time()
    
    
    # Hunt for a better solution
    gene = random.choice(classifier_names)
    orig_mut_rate = mutation_rate
    for generation in range(1,generations+1):
        # Make new weights using small mutation
        new_weights = classifier_weights
        mutation = (np.random.rand(1)-0.5)*mutation_rate
        
        new_weights[gene] += mutation
        #Constrain weights to [0,1]
        if new_weights[gene] < 0:
            new_weights[gene] = 0
        if new_weights[gene] > 1:
            new_weights[gene] = 1
            
        # Classify data and obtain new score
        ensemble_prediction = classifier_ensemble(multi_predictions,new_weights,groupby=groupby)
        y_pred = ensemble_prediction.surface
        new_score = sum(y_pred == y_true)/len(y_pred)
        change = new_score - score
        # Select fittest weights
        if change < 0: # If new score is worse change the gene, reestablish mutation rate
            scores.append(score)
            gene = random.choice(classifier_names)
            mutation_rate = orig_mut_rate
        elif change == 0: # Mutate, but don't change gene or the mutation rate
            scores.append(score)
            classifier_weights = new_weights
        elif change > 0: # Mutate and change mutation rate, but don't change the gene
            mutation_rate /= 0.75
            score = new_score
            scores.append(score)
            classifier_weights = new_weights
        
        # Update User
        progress = generation/generations
        time_elapsed = (time.time()-start)
        ETA = (time_elapsed/progress-time_elapsed)/60
        print('Score:{0:1.5f}\t|\tChange:{1:1.5f}\t|\tTime Remaining: {2:1.2f} min'.format(score,change,ETA), end = '\r')
    
    plt.plot(scores)
    return classifier_weights

In [ ]:
classifier_names = list(classifier_weights.keys())

In [ ]:
classifier_weights = CEParamSearch(train_filtered.surface,
                                   multi_predictions,
                                   classifier_names, 
                                   generations = 25,
                                   mutation_rate = 0.5,
                                   seed = 40)

In [ ]:
classifier_weights

The results of CEParamSearch are random, though they do tend towards optimization as you can see in the graph above. The classifier weights below were used for a late submission whose public and private score I reveal below.

classifier_weights = {'RFC': 1,
 'SVM': 1,
 'KNN': 0,
 'DT': 0.22006725,
 'XGB': 0.38377276}

Now that I have an optimized set of classifier weights I will run the unclassified dataset through the ensemble classifier to check its performance against Kaggle.

In [ ]:
multi_predictions_test = get_predictions(test_filtered,classifiers,scaler_f)

In [ ]:
ensemble_prediction_test = classifier_ensemble(predictions=multi_predictions_test,
                                 classifier_weights=classifier_weights)

A private score of 0.8280 and a public score of 0.4656 is good given that I'm not leveraging all of the position data. The public score mostly benefits from the position matched data.

<a id='section8'></a>
# Hybrid Classification
It's unfair to say that the previous ensemble classification was not a hybrid classification already. Hybrid classifiers are those that classify over multiple iterations, rather than classifying by comparing the output of different processes. The above ensemble classification was built on the existing classification that was done to select features, and the classification to find groups. This following hybrid classification leverages all of that information to its extreme. I will accept the classification that was applied after finding groups, use the classified features to obtain outputs from individual classifiers, classify the remaining unclassified data using the ensemble classifier I generated, and finally stitch the position matched classification with the ensemble matched classification.

In [ ]:
metadata = train_filtered.groupby('series_id').max()[['surface','test','new_group_id', 'orig_id']]
filtered_extracted_matched = metadata.join(extracted_features[feature_columns])
unmatched_filtered = filtered_extracted_matched[filtered_extracted_matched.surface == 'no_surface']

In [ ]:
unmatched_filtered.fillna(0,inplace=True)
unmatched_filtered.replace(-np.inf,0,inplace=True)
unmatched_filtered.replace(-np.inf,0,inplace=True)

In [ ]:
multi_predictions_unmatched = get_predictions(unmatched_filtered, classifiers, scaler_f)
ensemble_prediction_unmatched = classifier_ensemble(multi_predictions_unmatched, classifier_weights)
unmatched_output = ensemble_prediction_unmatched[['orig_id','surface']].reset_index().drop('series_id', axis =1)

In [ ]:
matched_output = metadata[(metadata.test == 1)&(metadata.surface != 'no_surface')][['orig_id','surface']].reset_index().drop('series_id', axis =1)

In [ ]:
output = pd.concat([matched_output,unmatched_output])
output.to_csv('submissions/submission_hybrid_classified_v7.csv', 
              header = ['series_id','surface'], 
              index = False)

The results of the Kaggle submission are wonderful! **A private score of 0.8777 and a public score of 0.9876 yields a combined accuracy of 0.9272 on the whole dataset.** We already know that ~4% of the data was misclassified after the groups were stitched, so only about ~3% of the remaining unclassified data was misclassified by the meta ensemble classifier. Overall, I'm thrilled with this score! To improve these scores I could tune all of the individual classifiers, search for more features to extract, and improve and the grouping mechanism.

<a id='section9'></a>
# Conclusion

The hybrid classifier that I developed was clearly better than using classical or ensemble classification schemes by themselves. Even a meta-ensemble of the standard classification schemes significantly outperforms a single classification scheme, which is helpful knowledge for future classification problems. Building this meta-classifier is simple enough, and tuning its performance even for a large number of parameters can be done using an evolutionary approach.